## User Management (Keycloak)
Interaction with Keycloak using Keycloak's python client and Identity API.

In [ ]:
import jwt
import json
from getpass import getpass
from keycloak import KeycloakPostError
from identityutils.keycloak_client import KeycloakClient
import requests
import urllib3

urllib3.disable_warnings()

base_domain = "develop.eoepca.org"
platform_domain = f"identity.keycloak.{base_domain}"
server_url = f"https://{platform_domain}"
realm = "master"
dummy_service_url = f"https://identity.dummy-service.{base_domain}"
identity_api_url = f"https://identity.api.{base_domain}"

## Client
We instantiate an object to interact with the Keycloak.

In [ ]:
admin_password = getpass("Admin password: ")
keycloak = KeycloakClient(
    server_url=server_url,
    realm=realm,
    username="admin",
    password=admin_password,
)

## User Authentication
User authenticates and the client receives an ID Token (JWT) that represents the user, and is used to identify the user in UMA authorization flows.

#### Create Users
Create two users, an Eric and Alice both with user role.

In [ ]:
eric_id = keycloak.create_user("eric", "eric", ["user"])
print("Created Eric user with id: " + eric_id)
alice_id = keycloak.create_user("alice", "alice", ["user"])
print("Created Alice user with id: " + alice_id)

#### Inspect Eric User Token

In [ ]:
token = keycloak.get_user_token("eric", "eric")
print("Eric token:\n" + json.dumps(token, indent = 2))
eric_access_token = token["access_token"]
jwt_header = jwt.get_unverified_header(eric_access_token)
print("JWT Header:\n" + json.dumps(jwt_header, indent = 2))
jwt_payload = jwt.decode(eric_access_token, options={"verify_signature": False})
print("JWT Payload:\n" + json.dumps(jwt_payload, indent = 2))

#### Inspect Alice User Token

In [ ]:
token = keycloak.get_user_token("alice", "alice")
print("Eric token:\n" + json.dumps(token, indent = 2))
alice_access_token = token["access_token"]
jwt_header = jwt.get_unverified_header(alice_access_token)
print("JWT Header:\n" + json.dumps(jwt_header, indent = 2))
jwt_payload = jwt.decode(alice_access_token, options={"verify_signature": False})
print("JWT Payload:\n" + json.dumps(jwt_payload, indent = 2))

The ID Token (JWT) identifies the user via user_name / sub (Subject) fields, and the client via the aud (Audience) field. The JWT is signed and can be verified, using the kid (Key ID) field, via the JWKS endpoint of the Authorization Server.

## Protect resources
Access a protected resources using UMA flow.

#### Assign premium role to Eric
Roles are used to define policies that will protect resources based on roles - Role-based access control (RBAC)

In [ ]:
realm_role = keycloak.create_realm_role('user-premium')
print("Created realm role: " + realm_role)
keycloak.assign_realm_roles_to_user(eric_id, realm_role)
print("Assigned " + realm_role + " role to Eric")

#### Register client
Register demo client

In [ ]:
client_id = "demo"
client_secret= "mpmhQOGEG4ocamf49HomRjPxILfpgVs6"
client_payload = {
    "clientId": client_id,
    "secret": client_secret,
    "name": "Demo client",
    "description": "Client used on demo notebook",
    "serviceAccountsEnabled": True,
    "directAccessGrantsEnabled": True,
    "authorizationServicesEnabled": True
}
keycloak.create_client(client_payload, skip_exists=True)
print("Created client: demo")

## Protect URIs
Right now, resources are protected by a default policy, which grants access to users within the realm.
Let's see how Keycloak protects resources using role based and user based policies.

#### Register resources
Register

In [ ]:
resources = [
    {
        "name": "Premium resource",
        "uri": "/protected/premium/*"
    },
    {
        "name": "Eric space",
        "uri": "/eric/*"
    },
    {
        "name": "Alice space",
        "uri": "/alice/*"
    }
]
keycloak.register_resources(client_id, resources, skip_exists=True)

#### Register policies
Register role based and user based policies

In [ ]:
policy = {
    "name": 'Only Premium User Policy',
    "roles": [
        {
            "id": "user-premium"
        }
    ]
}
p = keycloak.register_role_policy(client_id, policy, skip_exists=True)
print("Only Premium User Policy:")
print(p)

policy = {
    "name": 'Only Eric User Policy',
    "users": [eric_id]
}
p = keycloak.register_user_policy(client_id, policy)
print("Only Eric User Policy:")
print(p)

policy = {
    "name": 'Only Alice User Policy',
    "users": [alice_id]
}
p = keycloak.register_user_policy(client_id, policy)
print("Only Alice User Policy:")
print(p)

#### Register resource permissions
Resources permissions are set by assigning policies to resources.

In [ ]:
permissions = [
    {
        "name": "Premium permission",
        "type": "resource",
        "logic": "POSITIVE",
        "decisionStrategy": "UNANIMOUS",
        "resources": [
            "Premium resource"
        ],
        "policies": [
            "Only Premium User Policy"
        ]
    },
    {
        "name": "Eric space permission",
        "type": "resource",
        "logic": "POSITIVE",
        "decisionStrategy": "UNANIMOUS",
        "resources": [
            "Eric space"
        ],
        "policies": [
            "Only Eric User Policy"
        ]
    },
    {
        "name": "Alice space permission",
        "type": "resource",
        "logic": "POSITIVE",
        "decisionStrategy": "UNANIMOUS",
        "resources": [
            "Alice space"
        ],
        "policies": [
            "Only Alice User Policy"
        ]
    }
]
r = keycloak.assign_resources_permissions(client_id, permissions, skip_exists=True)
print(r)

#### Get PAT (Protection API token)
PAT (Protection API token) is used to access Keycloak's Protection API, which manages resources and policies.

In [ ]:
pat = keycloak.generate_protection_pat(client_id, client_secret)
print(json.dumps(pat, indent=2))
access_token = pat['access_token']

### Get Resource Ids

In [ ]:
# it's possible to query resources by many fields, including name and uri
premium_resource_id = keycloak.get_resource_id(client_id, client_secret, name="Premium resource")[0]
print("Premium resource: " + premium_resource_id)
eric_resource_id = keycloak.get_resource_id(client_id, client_secret, uri="/eric/*")[0]
print("Eric resource: " + eric_resource_id)
alice_resource_id = keycloak.get_resource_id(client_id, client_secret, uri="/alice/*")[0]
print("Alice resource: " + alice_resource_id)

#### Get UMA access token for eric space resource for both Eric and Alice

In [ ]:
uma_ticket = keycloak.create_permission_ticket(client_id, client_secret, resources=[eric_resource_id])['ticket']
print('UMA ticket for resource ' + eric_resource_id + ':\n' + uma_ticket)
eric_access_token = keycloak.get_user_token("eric", "eric", client_id=client_id, client_secret=client_secret)['access_token']
eric_rpt = keycloak.get_rpt(client_id, client_secret, "/eric/*", eric_access_token, uma_ticket)['access_token']
print('Eric RPT:\n' + str(eric_rpt))

In [ ]:
uma_ticket = keycloak.create_permission_ticket(client_id, client_secret, resources=[eric_resource_id])['ticket']
print('UMA ticket for resource ' + eric_resource_id + ':\n' + uma_ticket)
alice_access_token = keycloak.get_user_token("alice", "alice", client_id=client_id, client_secret=client_secret)['access_token']
try:
    alice_uma_access_token = keycloak.get_rpt(client_id, client_secret, "/eric/*", alice_access_token, uma_ticket)['access_token']
except KeycloakPostError as e:
    print(str(e))

Trying to get a UMA token for Alice results in a 403 Forbidden Error. The reason being Alice is not allowed to access the `/eric/*` resource because it's protected for only `eric` user.

## Identity API

#### Get Resources

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
url = identity_api_url + "/resources?client_id=" + client_id + "&client_secret=" + client_secret
print("GET " + url)
response = requests.get(url, headers=headers)
try:
    print(json.dumps(response.json(), indent=2))
except:
    print(response)

#### Get resource by id

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
resource_id = keycloak.get_resource_id(client_id, client_secret, name="Default Resource")[0]
url = identity_api_url + "/resources/" + resource_id + "?client_id=" + client_id + "&client_secret=" + client_secret
print("GET " + url)
response = requests.get(url, headers=headers)
print(json.dumps(response.json(), indent=2))

#### Register resource

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
data = [
    {
        'name': 'A resource',
        'uris': ["/protected/*"],
        "permissions": {
            "authenticated": True
        },
    }
]
url = f"{identity_api_url}/{client_id}/resources"
response = requests.post(url, json=data, headers=headers)
print("POST " + url)
print(str(response.status_code))
print(json.dumps(response.json(), indent=2))

#### Update resource

In [ ]:
resource_id = keycloak.get_resource_id(client_id, client_secret, name="A resource")[0]
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
data = {
    "name": "A resource",
    "uris": ["/protect/*"],
    "attributes": "attribute",
    "scopes": ["view"],
    "ownerManagedAccess": True,
    "displayName": "Display name"
}
url = f"{identity_api_url}/{client_id}/resources/{resource_id}?client_id={client_id}&client_secret={client_secret}"
response = requests.put(url + "/" + resource_id, json=data, headers=headers)
print("PUT " + url)
print(str(response.status_code))
print(str(response.json()))

#### Delete resource

In [ ]:
resource_id = keycloak.get_resource_id(client_id, client_secret, name="A resource")[0]
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
url = f"{identity_api_url}/{client_id}/resources/{resource_id}?client_id={client_id}&client_secret={client_secret}"
response = requests.delete(url, headers=headers)
print("DELETE " + url)
print(str(response.status_code))
print(str(response.json()))

#### Get client Policies

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
url = identity_api_url + "/" + client_id + "/policies"
response = requests.get(url, headers=headers)
print("GET " + url)
print(str(response.status_code))
print(json.dumps(response.json(), indent=2))

#### create client Policy

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
data = {
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "My Policy",
    "clients": [
        client_id
    ],
    "description": "Client policy"
}
url = identity_api_url + "/" + client_id + "/policies/client"
response = requests.post(url, json=data, headers=headers)
print("POST " + url)
print(str(response.status_code))
print(json.dumps(response.json(), indent=2))

#### Create Aggregated policy

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
data = {
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "Aggregated policy",
    "policies": ["My Policy"],
    "description": "Policy description"
}
url = identity_api_url + "/" + client_id + "/policies/aggregated"
response = requests.post(url, json=data, headers=headers)
print("POST " + url)
print(str(response.status_code))
print(json.dumps(response.json(), indent=2))

#### Create scope policy

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
data = {
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "Scope Policy",
    "scopes": [
        "view"
    ],
    "description": "Policy description"
}
url = identity_api_url + "/" + client_id + "/policies/scope"
response = requests.post(url, json=data, headers=headers)
print("POST " + url)
print(str(response.status_code))
print(json.dumps(response.json(), indent=2))

#### Create group policy

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
data = {
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "Group policy",
    "groups": ["0b86ce51-7027-4958-bac2-2d6af9ac3fbf"],
    "groupsClaim": "Groups claim",
    "description": "description"
}
url = identity_api_url + "/" + client_id + "/policies/group"
response = requests.post(url, json=data, headers=headers)
print("POST " + url)
print(str(response.status_code))
print(json.dumps(response.json(), indent=2))

#### - Create regex Policy

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
data = {
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "Regex policy",
    "pattern": ".*",
    "targetClaim": "preferred_username",
    "description": "Match all usernames"
}
url = identity_api_url + "/" + client_id + "/policies/regex"
response = requests.post(url, json=data, headers=headers)
print("POST " + url)
print(str(response.status_code))
print(json.dumps(response.json(), indent=2))

#### Create role policy

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
data = {
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "Role policy",
    "roles": [
        {
            "id": "user-premium",
            "required": False
        }
    ],
    "description": "Role policy"
}
url = identity_api_url + "/" + client_id + "/policies/role"
response = requests.post(url, json=data, headers=headers)
print("POST " + url)
print(str(response.status_code))
print(json.dumps(response.json(), indent=2))

#### create time policy

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
data = {
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "Time policy",
    "description": "description",
    "year": 2023,
    "yearEnd": 2024
}
url = identity_api_url + "/" + client_id + "/policies/time"
response = requests.post(url, json=data, headers=headers)
print("POST " + url)
print(str(response.status_code))
print(json.dumps(response.json(), indent=2))

#### Create user policy

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
data = {
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "User policy",
    "users": [
        "eric"
    ],
    "description": "test"
}
url = identity_api_url + "/" + client_id + "/policies/user"
response = requests.post(url, json=data, headers=headers)
print("POST " + url)
print(str(response.status_code))
print(json.dumps(response.json(), indent=2))
policy_id = response.json()['id']

#### Update policies
Change previous added user policy from users "eric" to "alice"

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
data = {
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "User policy",
    "description": "description",
    "users": [
        "alice"
    ],
}
url = identity_api_url + "/" + client_id + "/policies/user/" + policy_id
response = requests.put(url, json=data, headers=headers)
print("PUT " + url)
print(str(response.status_code))
print(json.dumps(response.json(), indent=2))

#### Delete policies

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
url = identity_api_url + "/" + client_id + "/policies/" + policy_id
response = requests.delete(url, headers=headers)
print("DELETE " + url)
print(str(response.status_code))

#### Get client permissions

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
url = identity_api_url + "/" + client_id + "/permissions"
response = requests.get(url, headers=headers)
print("GET " + url)
print(str(response.status_code))
print(json.dumps(response.json(), indent=2))

#### Get client management permissions

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
url = identity_api_url + "/" + client_id + "/permissions/management"
response = requests.get(url, headers=headers)
print("GET " + url)
print(str(response.status_code))
print(json.dumps(response.json(), indent=2))


#### Get client resources permissions

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
url = identity_api_url + "/" + client_id + "/permissions/resources"
response = requests.get(url, headers=headers)
print("GET " + url)
print(str(response.status_code))
print(json.dumps(response.json(), indent=2))

#### Create client resources permissions

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
data = {
    "type": "resource",
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "Permission-Name 2",
    "resources": [
        "5bd655ec-2575-406e-aa08-28b1bd25f476"
    ],
    "policies": [
        "57d4a363-6b40-4dec-93e9-a46a1a8e492f"
    ]
}
url = identity_api_url + "/" + client_id + "/permissions/resources"
response = requests.get(url, json=data, headers=headers)
print("GET " + url)
print(str(response.status_code))
print(json.dumps(response.json(), indent=2))

#### Update client management permissions

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
data = {
    "enabled": True
}
url = identity_api_url + "/" + client_id + "/permissions/management"
response = requests.put(url, json=data, headers=headers)
print("PUT " + url)
print(str(response.status_code))
print(json.dumps(response.json(), indent=2))

#### Register and Protect a Resource

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
data = [
    {
        "name": "Eric resource23233",
        "uris": ["/eric/*"],
        "permissions": {
            "user": ["eric"]
        }
    },
    {
        "name": "Alice resource3233",
        "uris": ["/alice/*"],
        "permissions": {
            "user": ["alice"]
        }
    }
]
url = identity_api_url + "/" + client_id + "/resources"
response = requests.post(url, json=data, headers=headers)
print("POST " + url)
print(str(response.status_code))
print(json.dumps(response.json(), indent=2))

#### Delete Resource and its policies and permissions

In [ ]:
from urllib.parse import quote
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
resource_name = "Eric resource"
url = identity_api_url + "/" + client_id + "/resources/" + quote(resource_name) + "/all"
response = requests.delete(url, headers=headers)
print('DELETE ' + url)
print(response.status_code)
resource_name = "Alice resource"
url = identity_api_url + "/" + client_id + "/resources/" + quote(resource_name) + "/all"
response = requests.delete(url, headers=headers)
print('DELETE ' + url)
print(response.status_code)

#### Create client, create and protect resources in one endpoint

In [ ]:
access_token = keycloak.get_user_token("admin", admin_password)["access_token"]
headers = {
    "Authorization": "Bearer " + access_token
}
payload = {
    "clientId": "dummy-service",
    "name": "Dummy Service",
    "description": "Client used for Dummy service",
    "resources": [
        {
            "name": "Eric space",
            "uris": ["/eric/*"],
            "permissions": {
                "user": ["eric"]
            }
        },
        {
            "name": "Alice space",
            "uris": ["/alice/*"],
            "permissions": {
                "user": ["alice"]
            }
        }
    ]
}
url = identity_api_url + '/clients'
response = requests.post(url, json=payload, headers=headers)
print("POST " + url)
print(response.status_code)
print(response.text)

## Bash Script
Execute script to create a client with protected resources

In [ ]:
import subprocess
import urllib.request

with urllib.request.urlopen(
        "https://raw.githubusercontent.com/EOEPCA/um-identity-service/master/scripts/create-client.sh") as f:
    script = f.read().decode('utf-8')
    out = open("create-client.sh", "w")
    out.write(script)
    out.flush()

access_token = keycloak.get_user_token("admin", admin_password)["access_token"]

client_id = input("Client id: ")
client_name = "Test client"
client_description = "Test client from demo notebook"

script_args = ["create-client.sh",
                       "-e develop",
                       "-t " + access_token,
                       "--id=" + "\"" + client_id + "\"",
                       "--name=" + "\"" + client_name + "\"" ,
                       "--description=" + "\"" + client_description + "\"" ,
                       "--resource=\"Eric space\"", "--uris=/ericspace/*", "--users=eric",
                       "--resource=\"Alice space\"", "--uris=/alicespace/*", "--users=alice"
]
print("Executing script:\n" + ' '.join(script_args) + '\n')

#r = subprocess.run(script_args, shell=True, text=True, capture_output=True)
#print(r)